#### 协程（yield from）

##### 语法

In [ ]:
# yield from 简要实现
_i = iter(EXPR)  # __iter__ -> __await__
try:
    _y = _i.send(None)  # prime
except StopIteration as _e:
    _r = _e.value  # 直接就结束了，一次 yield 都没遇上
else:
    while 1:  # 不遇到 StopIteration 不算完
        try:
            _s = yield _y  # 照原样 yield 出去，并接受 send 传入的值
        except GeneratorExit as _e:  # 处理 close
            _i.close()
            raise _e
        except BaseException as _e:  # 处理其它异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
        else:
            try:
                _y = _i.send(_s)  # 接受到的值原样再 send 下去
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 出来的值就是结果

In [ ]:
# 已知总是会 send(None)，继续简化
_i = iter(EXPR)  # __iter__
while 1:  # 不遇到 StopIteration 不算完
    try:
        _y = _i.send(None)  # 总是 None，也就无所谓 prime
    except StopIteration as _e:
        _r = _e.value
        break
    else:
        try:
            yield _y  # 照原样 yield 出去，不再接受 send 传入的值，因为总是 None
        except GeneratorExit as _e:  # 处理 close
            _i.close()
            raise _e
        except BaseException as _e:  # 处理其它异常
            _x = sys.exc_info()
            try:
                _y = _i.throw(*_x)
            except StopIteration as _e:
                _r = _e.value
                break
RESULT = _r  # StopIteration 出来的值是结果

In [ ]:
# 再去掉异常处理部分代码
_i = iter(EXPR)  # __iter__
while 1:  # 不遇到 StopIteration 不算完
    try:
        _y = _i.send(None)  # 总是 None，也就无所谓 prime
    except StopIteration as _e:
        _r = _e.value
        break
    else:
        yield _y  # 照原样 yield 出去，不再接受 send 传入的值，因为总是 None
RESULT = _r  # StopIteration 出来的值就是结果

##### 阶段1（定义一个任务）
- 一个同步模式的简单任务

In [1]:
def one_task():
    print(f'begin task')
    ... # 其它步骤
    print(f' begin big_step:')
    
    big_result = big_step()  # <---
    
    print(f' end big_step with {big_result}')
    ... # 其它步骤
    
    print(f'end task')

def big_step():
    ... # 其它小步骤
    print(f'  begin small_step:')
    
    small_result = small_step()  # <---
    
    print(f'  end small_step with {small_result}')
    ... # 其它小步骤
    return small_result  # 123

def small_step():
    print('    努力工作中...')
    return 123  # 完成了


# 执行任务
one_task()

begin task
 begin big_step:
  begin small_step:
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


- 阻塞

In [2]:
from time import sleep

def small_step():
    print('      躺平中')
    sleep(2)  # 阻塞
    print('    努力工作中...')
    return 123  # 完成了

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


- 使用yield 使得协程不阻塞

In [3]:
from time import sleep

def small_step():
    print('      躺平中')
    yield sleep(2)  # 阻塞
    print('    努力工作中...')
    return 123  # 完成了

one_task()

begin task
 begin big_step:
  begin small_step:
  end small_step with <generator object small_step at 0x000001DCDA679690>
 end big_step with <generator object small_step at 0x000001DCDA679690>
end task


In [4]:
def big_step():
    ... # 其它小步骤
    print(f'  begin small_step:')
    
    small_coro = small_step()  # <---
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            small_result = e.value
            break
        else:
            pass
    
    print(f'  end small_step with {small_result}')
    ... # 其它小步骤
    return small_result  # 123

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


- 将阻塞由下游传到上游

In [5]:
from time import sleep

def small_step():
    print('      躺平中')
    yield (sleep, 2)  # 阻塞
    print('    努力工作中...')
    return 123  # 完成了

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


In [6]:
import time
from time import sleep

def small_step():
    print('      躺平中')
    t1 = time.time()
    yield (sleep, 2)  # 阻塞
    assert time.time() - t1 >= 2, "阻塞时间不足"
    print('    努力工作中...')
    return 123  # 完成了

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中


AssertionError: 阻塞时间不足

In [7]:
def big_step():
    ... # 其它小步骤
    print(f'  begin small_step:')
    
    small_coro = small_step()  # <---
    while True:
        try:
            x = small_coro.send(None)
        except StopIteration as e:
            small_result = e.value
            break
        else:
            yield x  # 将阻塞传到上游
    
    print(f'  end small_step with {small_result}')
    ... # 其它小步骤
    return small_result  # 123

one_task()

begin task
 begin big_step:
 end big_step with <generator object big_step at 0x000001DCDBB60190>
end task


In [8]:
def one_task():
    """一个任务"""
    print('begin task')
    ... # 其它步骤
    print(f' begin big_step:')

    # big_result = big_step()
    big_coro = big_step()
    while True:
        try:
            x = big_coro.send(None)
        except StopIteration as e:
            big_result = e.value
            break
        else:
            func, arg = x
            func(arg)

    print(f' end big_step with {big_result}')
    ... # 其它步骤

    print('end task')

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


- 总结
    - 协程本身并不能消除阻塞
    - 协程具有“传染性”
    - 协程通过yield 把阻塞换个方式传递给了上游
    - 最终阻塞仍然需要被解决

##### 阶段2（yield from）

In [9]:
def big_step():
    ... # 其它小步骤
    print(f'  begin small_step:')
    
    small_result = yield from small_step()
    # small_coro = small_step()  # <---
    # while True:
    #     try:
    #         x = small_coro.send(None)
    #     except StopIteration as e:
    #         small_result = e.value
    #         break
    #     else:
    #         pass
    
    print(f'  end small_step with {small_result}')
    ... # 其它小步骤
    return small_result  # 123

one_task()

begin task
 begin big_step:
  begin small_step:
      躺平中
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


将最末端的遇到阻塞而不得不主动 yield 的协程称之为「主动协程」
中间接受到下游的传导而不得不跟随着 yield 的协程称之为「被动协程」

- 「主动协程」是最先出栈的位置
- 「被动协程」可能有很多层
- yield from 大大简化了「被动协程」的编码


In [10]:
class YielfFromAble:
    def __init__(self, obj):
        self.value = obj
    
    def __iter__(self):
        yield self.value

In [11]:
import time
from time import sleep

def small_step():
    print('    休息一下，马上回来')
    t1 = time.time()
    
    yield from YielfFromAble((sleep, 2))
    
    assert time.time() - t1, '睡眠时间不足！'
    print('    努力工作中...')
    return 123  # 完成了...

one_task()

begin task
 begin big_step:
  begin small_step:
    休息一下，马上回来
    努力工作中...
  end small_step with 123
 end big_step with 123
end task


##### 阶段3

- 任务彻底协程化

In [12]:
def one_task():
    """一个任务"""
    print('begin task')
    ... # 其它步骤
    print(f' begin big_step:')

    # big_result = big_step()

    big_result = yield from big_step()

    # big_coro = big_step()
    # while True:
    #     try:
    #         x = big_coro.send(None)
    #     except StopIteration as e:
    #         big_result = e.value
    #         break
    #     else:
    #         func, arg = x
    #         func(arg)

    print(f' end big_step with {big_result}')
    ... # 其它步骤

    print('end task')

one_task()

<generator object one_task at 0x000001DCDBB61770>

- 一个通用的任务驱动器

In [13]:
class YielfFromAble:
    def __init__(self, obj):
        self.value = obj
    
    def __iter__(self):
        yield self

In [16]:
class Task:
    def __init__(self, coro):
        self.coro = coro  # 保存协程对象

    def run(self):
        print('任务开始运行')
        """驱动任务运行直到完成"""
        
        while True:
            try:
                x = self.coro.send(None)
            except StopIteration as e:
                result =  e.value  # 返回最终结果
                break
            else:
                assert isinstance(x, YielfFromAble)
                func, arg = x.value
                func(arg)
        print('任务运行结束')

In [17]:
t = Task(one_task())
t.run()

任务开始运行
begin task
 begin big_step:
  begin small_step:
    休息一下，马上回来
    努力工作中...
  end small_step with 123
 end big_step with 123
end task
任务运行结束
